In [10]:
# Import libraries
import pandas as pd
import numpy as np
import openai
from openai import OpenAI
from IPython.display import display, Markdown
import os
 


# Define absolute python path
import sys
sys.path.insert(0, '/Users/giorgiobolchi2/Documents/JRC/LLM') 


In [ ]:
# Set API token
TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6ImY3NDcxY2MwLTE5NWItNDZhOS1hNjc1LTM0Y2EzY2E1MmI2YSIsImlzcyI6ImdwdGpyYyIsImlhdCI6MTczNzQ1NjU4NywiZXhwIjoxNzUwNDY0MDAwLCJpc19yZXZva2VkIjpmYWxzZSwiYWNjb3VudF9pZCI6IlFXeGxjeTVTUlVKRlEwQmxlSFF1WldNdVpYVnliM0JoTG1WMVMyNXZkMU5FUjNNPSIsInVzZXJuYW1lIjoiQWxlcy5SRUJFQ0BleHQuZWMuZXVyb3BhLmV1IiwicHJvamVjdF9pZCI6Iktub3dTREdzIiwiZGVwYXJ0bWVudCI6IkpSQy5ELjEiLCJxdW90YXMiOlt7Im1vZGVsX25hbWUiOiJncHQtNG8iLCJleHBpcmF0aW9uX2ZyZXF1ZW5jeSI6ImRhaWx5IiwidmFsdWUiOjQwMDAwMH0seyJtb2RlbF9uYW1lIjoiZ3B0LTM1LXR1cmJvLTExMDYiLCJleHBpcmF0aW9uX2ZyZXF1ZW5jeSI6ImRhaWx5IiwidmFsdWUiOjQwMDAwMH0seyJtb2RlbF9uYW1lIjoiZ3B0LTQtMTEwNiIsImV4cGlyYXRpb25fZnJlcXVlbmN5IjoiZGFpbHkiLCJ2YWx1ZSI6NDAwMDAwfV0sImFjY2Vzc19ncm91cHMiOlt7ImlkIjoiNDJkMTUzMTAtNzdlZS00MDgyLTk1NzQtZDZhYTk5YzliZDgzIiwiYWNjZXNzX2dyb3VwIjoiZ2VuZXJhbCJ9XX0.0S3jlmjrBLGPwQaJ4U5F20DpacvyDcK_NsS412BX_Pw"
openai.api_base = "https://api-gpt.jrc.ec.europa.eu/v1"
openai.api_key = TOKEN

client = OpenAI(
        api_key = TOKEN,
        base_url = "https://api-gpt.jrc.ec.europa.eu/v1/"    )

In [ ]:
# Define request function 
def run_API_request(prompt):

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gpt-4o",
    )

    return chat_completion.choices[0].message.content



In [ ]:
# Import NACE data from NACEdata.py file

from Data.NACEdata import NACElevel0, NACElevel1, NACElevel2, NACElevel3     

# Import a subset of target data
df = pd.read_excel('/Users/giorgiobolchi2/Documents/JRC/LLM/Data/XLSX_target_data_v1.2_LLMsubsetxlsx.xlsx')
target_list = str(df['Target'].to_list())

# doublec check file formats so that they're compatible with the run_API_request function
print(type(target_list))
print(type(NACElevel0))
print(type(NACElevel1))
print(type(NACElevel2))
print(type(NACElevel3))



In [ ]:

prompt = f'''Hello,

            Data input: get acquainted with the following data:
            - NACE classification categories:  {NACElevel1} + {NACElevel2} +{NACElevel3}.
            - List of targets: {target_list}.
            
            Task: 
            - For each target, analyse its content description and assign to each target a NACE category for each level (0,1,2,3). 

            Answer format: provide your answer as a table in csv format please (separator: ";"), with the following columns:
            - Target code (e.g., TA1.9)
            - Target content (e.g., The contribution of the sectors covered by the EU ETS with respect to the EU Climate ambition should be of -62 % compared to 2005 (increasing the linear emissions reduction factor from 2.2 % per year up to 4.4 %)) 
            - NACE_level1 (e.g., D - Electricity, Gas, Steam and Air Conditioning Supply)
            - NACE_level1_extra1 (e.g.if other categories overlap)
            - NACE_level1_extra2 (e.g.if other categories overlap)
            - NACE_level2 (e.g., D35 - Electricity, gas, steam and air conditioning supply)
            - NACE_level2_extra1 (e.g.if other categories overlap)
            - NACE_level2_extra2 (e.g.if other categories overlap)
            - NACE_level_3 (e.g., D35.1 - Electric power generation, transmission and distribution)
            - NACE_level_3_extra1 (e.g. if other categories overlap)
            - NACE level3_extra2 (e.g. if other categories overlap)
            - Justification
            - Confidence (e.g. confidence value from 0 to 10 about the assignation choices that are made)

            Specifications:
            - If there is some overlap, add the multiple categories possibilites (up to maximum 3 per NACE level). 
            - Include the name of the NACE categories.
            - Don't forget to provide the title of the target. If there is no target content, do not invent new content, just state it as empty.
            - For each target, write one or max 2 sentences justifying your choice.
            - Output only the csv table and no additional commentary text.

            Thank you.'''

# generate answer
answer= run_API_request(prompt)


In [ ]:
#print(answer)

#print answer in Markdown format
display(Markdown(answer))


In [ ]:
# Export raw text as csv directly
output_path = "/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/"
output_name = "output.csv"

with open((os.path.join(output_path, output_name)), 'w') as f:
    f.write(answer)


In [24]:

# Load the file
file= '/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/0131/clean_trial/0131output_TA1.1_sNone_t0.2.csv'

with open(file, 'r') as file:
    lines = file.readlines()


# Remove the first and last line
lines = lines[1:-1]

# Save the updated lines to a new text file
with open('/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/0131/clean_trial/0131output_TA1.1_sNone_t0.2.csv', 'w') as file:
    file.writelines(lines)


In [25]:
date ='0131'
output_directory = f'/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/{date}/clean_trial/'



for filename in os.listdir(output_directory):

    # Check if the file name contains the characters 'TA'
    if 'TA' in filename and '.csv' in filename:
        
        # Open the file for reading and writing
        with open(f'{output_directory}{filename}', 'r+') as file:  # r+ = mode to open the file for both reading and writing
            lines = file.readlines()
            lines = lines[1:-1] # remove first and last row
            file.seek(0)  # move the file pointer to the beginning
            file.writelines(lines) # write the modified lines back to the file.
            file.truncate() # set the length of the file to the current position of the file pointer.


In [7]:
import pandas as pd


file_name = 'estat_nama_10_a64_e'
directory = f'/Users/giorgiobolchi2/Documents/JRC/LLM/Data/ESTAT/'


tsv = pd.read_csv(f'{directory}{file_name}.tsv', sep='\t')
tsv.to_csv(f'{directory}{file_name}.csv', sep=';', index=True)